In [1]:
import os
import sys
import numpy as np
import csv
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import pickle

In [2]:
df_train = pd.read_pickle("./train_mr.pkl")
df_test = pd.read_pickle("./test_mr.pkl")
pd.set_option('display.max_columns', 500)

In [3]:
df_train.head()

,acquirer,bank,card,money,trade_cat,coin,online,trade_type,fallback,3ds,fraud_ind,pay_type,install,term,date,time,mcc,shop,excess,city,nation,status,key
0,6716,12765,101514,966.580017,4,62,0,2,0.0,0.0,0,5,1,2,1,29.0,247,34949,0,5817,102,0,1119159
1,5575,34837,60869,1071.099976,5,62,1,8,0.0,0.0,0,5,0,0,1,40.0,203,33794,0,2138,44,0,641444
2,6769,22630,110573,438.209991,5,62,0,5,0.0,0.0,0,5,0,0,1,45.0,251,79257,0,3588,102,0,188670
3,6767,91588,208961,1174.170044,5,62,0,5,0.0,0.0,0,5,0,0,1,48.0,293,14765,0,5817,102,0,1475737
4,3198,84148,180103,367.290009,5,60,0,5,0.0,0.0,0,5,0,0,1,51.0,251,68966,0,3982,38,0,641443


# Feature Engineering

## 1

In [4]:
#money / term groupby high_category: max, min, mean, std, var, median, median_diff, quantiles
def money_term_inf(dataframe,c,b):
    #b grouped by c
    a = dataframe.copy()
    a[b + '_' + c + '_max'] = a[c].map(a.loc[:, [b, c]].groupby(c).max().loc[:, b].to_dict())
    a[b + '_' + c + '_min'] = a[c].map(a.loc[:, [b, c]].groupby(c).min().loc[:, b].to_dict())
    a[b + '_' + c + '_mean'] = a[c].map(a.loc[:, [b, c]].groupby(c).mean().loc[:, b].to_dict())
    a[b + '_' + c + '_std'] = a[c].map(a.loc[:, [b, c]].groupby(c).std().loc[:, b].to_dict())
    a[b + '_' + c + '_var'] = a[b + '_' + c + '_std']**2
    a[b + '_' + c + '_median'] = a[c].map(a.loc[:, [b, c]].groupby(c).median().loc[:, b].to_dict())
    a[b + '_' + c + '_median_diff'] = a[b] - a[b + '_' + c + '_median']
    return a

## 2

In [5]:
#card_num: one bank has how many cards. ( 一個帳號有幾張卡 )
def card_num(dataframe):
    a = dataframe.copy()
    a['card_num'] = a['bank'].map(a.loc[:, ['card', 'bank']].groupby('bank').count().loc[:,'card'].to_dict())
    return a

## 4

In [11]:
#dominant coin type 貨幣在那個國家交易過幾次
def coin_trade_country_per(dataframe):
    a = dataframe.copy()
    dct =(a.groupby(['nation','coin']).size()/a.groupby('nation').size()).to_dict()
    a['coin_country_per'] = a.set_index(['nation', 'coin']).index.map(dct.get)
    return a

## 9

In [7]:
#money / term => term_money 分期金額
def money_divide_term(dataframe):
    a = dataframe.copy()
    a['money_term'] = a['money']/(a['term']+1)
    return a

## 11

In [26]:
#一個國家有幾個城市
def count_city_num(dataframe):
    a = dataframe.copy()
    a['city_num'] = a['nation'].map(a.loc[:, ['city', 'nation']].groupby('nation').count().loc[:,'city'].to_dict())
    return a

## 12

In [43]:
#nation/ acquirer/bank， fallback / online/ 3ds /install/excess 的交易數量比例
def trade_per(dataframe,c,b):
    #b grouped by c
    a = dataframe.copy()
    dct =(a.groupby([c,b]).size()/a.groupby(c).size()).to_dict()
    a[b + '_' + c + '_tradeper']  = a.set_index([c, b]).index.map(dct.get)
    return a

## 15

In [36]:
#week day (date 除以 7 餘數)
def whatday(dataframe):
    a = dataframe.copy()
    a['week'] = a['date']%7
    return a

# 跑Function

In [8]:
#跑money_term_inf
high_category = ['acquirer', 'bank', 'card', 'coin', 'mcc', 'shop', 'city', 'nation']
for i in (high_category):
    df_train  =money_term_inf(df_train,i,'money')
    df_train = money_term_inf(df_train,i,'term')

In [9]:
#跑card_num
df_train = card_num(df_train)

In [12]:
#跑coin_trade_country_per
df_train = coin_trade_country_per(df_train)

In [13]:
#跑money_divide_term
df_train = money_divide_term(df_train)

In [27]:
#跑count_city_num
df_train = count_city_num(df_train)

In [37]:
#跑count_city_num
df_train = whatday(df_train)

In [46]:
#跑trade_per
binary_category = ['fallback','online','3ds','install','excess']
#可自定義
group_category = ['nation','acquirer','bank']
for i in group_category:
    for j in binary_category:
        df_train = trade_per(df_train,i,j)